运行OpenQASM量子线路

In [2]:
import os

from qutrunk.circuit import QCircuit

BASE_DIR = os.getcwd()
file_path = BASE_DIR + '/qutrunk/example/allgates.qasm'

circuit = QCircuit.from_qasm_file(file_path)
circuit.print_qasm()
result = circuit.run(shots=100)
print(result.get_counts())

OPENQASM 2.0;
include "qulib1.inc";
qreg q[5];
creg c[5];
h q[0];
h q[0];
h q[1];
cx q[0],q[1];
x q[0];
ccx q[0],q[1],q[2];
r(1.5707963267948966,1.5707963267948966) q[0];
s q[0];
sdg q[0];
t q[0];
tdg q[0];
x1 q[0];
y1 q[0];
rx(1.5707963267948966) q[1];
ry(1.5707963267948966) q[1];
rz(1.5707963267948966) q[1];
swap q[0],q[1];
iswap(1.5707963267948966) q[0],q[1];
sqrtswap q[0],q[1];
cx q[0],q[1];
cy q[0],q[1];
cz q[0],q[1];
cp(1.5707963267948966) q[0],q[1];
cr(1.5707963267948966) q[0],q[1];
crx(1.5707963267948966) q[0],q[1];
cry(1.5707963267948966) q[0],q[1];
crz(1.5707963267948966) q[0],q[1];
p(1.5707963267948966) q[2];
x q[2];
y q[2];
z q[2];
ccx q[0],q[1],q[2];
ccz q[0],q[1],q[2];
rxx(1.5707963267948966) q[0],q[1];
ryy(1.5707963267948966) q[0],q[1];
rzz(1.5707963267948966) q[0],q[1];
u1(1.5707963267948966) q[0];
u2(1.5707963267948966,1.5707963267948966) q[0];
u3(1.5707963267948966,1.5707963267948966,1.5707963267948966) q[0];
cu(1.5707963267948966,1.5707963267948966,1.5707963267948966

解析并运行QuSL量子线路

In [4]:
import os

from qutrunk.circuit import QCircuit

BASE_DIR = os.getcwd()
file_path = BASE_DIR + '/qutrunk/example/bell_pair.qusl'

circuit = QCircuit.from_qusl_file(file_path)
circuit.print()
result = circuit.run(shots=100)
print(result.get_counts())

qreg q[2]
creg c[2]
H * q[0]
MCX(1) * (q[0], q[1])
Measure * q[0]
Measure * q[1]
[{"00": 50}, {"11": 50}]


使用qusprout作为后端运行量子线路

In [3]:
from qutrunk.circuit import QCircuit
from qutrunk.backends import BackendQuSprout
from qutrunk.circuit.gates import H, CNOT, Measure

# allocate
qc = QCircuit(backend=BackendQuSprout())
qr = qc.allocate(2)

# apply gate
H * qr[0]
CNOT * (qr[0], qr[1])
Measure * qr[0]
Measure * qr[1]

# run circuit
res = qc.run(shots=100)

# print result
print(res.get_counts())

[{"00": 53}, {"11": 47}]


使用IBM后端运行量子线路

result== [{'qubits': [0], 'name': 'u2', 'params': [0, 3.141592653589793]}, {'qubits': [0, 1], 'name': 'cx'}, {'qubits': [0], 'name': 'measure', 'memory': [0]}, {'qubits': [1], 'name': 'measure', 'memory': [1]}]
- Authenticating...
user API token: 247891ade16963963eb432d5ae4c7bbd1948d893f256a9f5d94af94628c5b57c73877dbf6ad4d2bd0ffc0e61d6aa001897666f1d75c3e613784ed8f2c7cafe68
- List of IBMQ devices available:
{'ibmq_qasm_simulator': {'nq': 32, 'coupling_map': None, 'version': '0.1.547'}, 'ibmq_lima': {'nq': 5, 'coupling_map': [[0, 1], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 4], [4, 3]], 'version': '1.0.38'}, 'ibmq_belem': {'nq': 5, 'coupling_map': [[0, 1], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 4], [4, 3]], 'version': '1.0.48'}, 'ibmq_quito': {'nq': 5, 'coupling_map': [[0, 1], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 4], [4, 3]], 'version': '1.1.33'}, 'simulator_statevector': {'nq': 32, 'coupling_map': None, 'version': '0.1.547'}, 'simulator_mps': {'nq': 100, 'coupling_map': N

以OpenQASM格式导出量子线路

In [6]:
from qutrunk.circuit import QCircuit
from qutrunk.backends import BackendIBM
from qutrunk.circuit.gates import H, CNOT, Measure

# allocate
qc = QCircuit()
qr = qc.allocate(2)

# apply gate
H * qr[0]
CNOT * (qr[0], qr[1])
Measure * qr[0]
Measure * qr[1]

# export
qc.print_qasm("bell_pair.qasm")

with open(file="bell_pair.qasm") as f:
    print(f.read())

OPENQASM 2.0;
include "qulib1.inc";
qreg q[2];
creg c[2];
h q[0];
cx q[0],q[1];
measure q[0] -> c[0];
measure q[1] -> c[1];



应用振幅放大：先对4个量子比特做均匀叠加，然后选择状态值为7作为标记值，做3次QAA迭代计算，运行后得到的结果是7对应3状态出现的概率超过了96%

In [1]:
from qutrunk.circuit.ops import QAA
from qutrunk.circuit import QCircuit
from qutrunk.circuit.gates import H, All

circuit = QCircuit()
qureg = circuit.allocate(4)
All(H) * qureg
QAA(3, 7) * qureg
for i in range(2 ** len(qureg)):
    print(circuit.get_prob_amp(i))

0.002578735351562488
0.0025787353515624874
0.002578735351562488
0.0025787353515624874
0.002578735351562488
0.0025787353515624874
0.002578735351562488
0.9613189697265575
0.002578735351562488
0.0025787353515624874
0.002578735351562488
0.0025787353515624874
0.002578735351562488
0.0025787353515624874
0.002578735351562488
0.0025787353515624853


使用QSP制备量子状态

In [1]:
from qutrunk.circuit.ops import QSP
from qutrunk.circuit import QCircuit
from qutrunk.circuit.gates import H, All, Measure

circuit = QCircuit()
qureg = circuit.allocate(2)
QSP("+") * qureg
circuit.print(unroll=False)
print(circuit.get_all_state())

qreg q[2]
creg c[2]
QSP('+') * q
['0.50000000000000, 0.00000000000000', '0.50000000000000, 0.00000000000000', '0.50000000000000, 0.00000000000000', '0.50000000000000, 0.00000000000000']


申请多个量子寄存器

In [1]:
from qutrunk.circuit import QCircuit
from qutrunk.circuit.gates import H, CNOT, Measure

qc = QCircuit()
# allocate 2 qureg: q1, q2
q1, q2 = qc.allocate([1, 1])

H * q1[0]
CNOT * (q1[0], q2[0])
Measure * q1[0]
Measure * q2[0]
res = qc.run(shots=100)
print(q1.to_cl())
print(q2.to_cl())
print(res.get_counts())

[1]
[1]
[{"00": 50}, {"11": 50}]


自定义量子门

In [1]:
from qutrunk.circuit import QCircuit
from qutrunk.circuit.gates import H, CNOT, gate, All, Measure

@gate
def my_gate(q):
    H * q[0]
    CNOT * (q[0], q[1])

circuit = QCircuit()
q = circuit.allocate(2)
my_gate * q
All(Measure) * q
res = circuit.run(shots=100)
print(res.get_counts())


[{"00": 50}, {"11": 50}]
